In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json

In [2]:
ners = ["DISEASE","EFFECT"]

In [4]:
l = []
for ner in ["DISEASE"]:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [6]:
df = df.reset_index().drop(columns=["index"])

In [7]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[The, current, study, did, show, that, alanine...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.891744613647461, -3.85470700263977, 6.500...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",DISEASE
1,"[The, results, are, presented, as, a, phylogen...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.951909065246582, -3.7594962120056152, 6.2...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11...",DISEASE
2,"[The, virulent, serotype, 2, ST7, strain, SC84...","[O, O, O, O, O, O, O, O, O, O, O, B, I, I, I, ...",2,"[[-2.828230142593384, -3.822613000869751, 6.24...","[-100, 2, 2, 2, 2, 2, -100, 2, 2, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10,...",DISEASE
3,"[A, human, stool-derived, Bilophila, wadsworth...","[O, O, O, O, O, O, O, B, I, O, O, O, O, O, O, O]",3,"[[-2.458846092224121, -3.563410997390747, 6.05...","[-100, 2, 2, 2, -100, -100, 2, -100, 2, -100, ...","[None, 0, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 6, ...",DISEASE
4,"[The, L-H, group, showed, significantly, highe...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.737249374389648, -3.846871137619018, 6.35...","[-100, 2, 2, -100, -100, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,...",DISEASE
...,...,...,...,...,...,...,...
604,"[S., citri, strain, BLH-13, was, isolated, in,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",604,"[[-2.632442951202392, -3.8172717094421382, 6.4...","[-100, 2, -100, 2, -100, 2, 2, -100, -100, -10...","[None, 0, 0, 1, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, ...",DISEASE
605,"[Nonetheless, ,, ADO, genes, with, high, cover...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",605,"[[-2.719882726669311, -4.03209114074707, 6.377...","[-100, 2, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...",DISEASE
606,"[In, particular, ,, the, combination, of, 5‐FU...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",606,"[[-2.6313037872314453, -4.045229434967041, 5.8...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",DISEASE
607,"[The, average, colony, growth, rate, (, 10.72,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",607,"[[-2.942236423492431, -3.877107381820678, 6.71...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",DISEASE


In [48]:
label_ids = df[["label_ids"]]

In [49]:
for j in range(len(df)):
	for i in zip(df.loc[j,"tokens"],df.loc[j,"ner_tags"],df.loc[j,"label_ids"]):
		print(i)

('We', 'O', -100)
('started', 'O', 2)
('to', 'O', 2)
('investigate', 'O', 2)
('the', 'O', 2)
('growth', 'O', 2)
('factor-like', 'O', 2)
('activity', 'O', 2)
('of', 'O', -100)
('TpF1', 'O', -100)
(',', 'O', 2)
('a', 'O', 2)
('major', 'O', 2)
('antigen', 'O', -100)
('expressed', 'O', 2)
('by', 'O', 2)
('T.', 'B', 2)
('pallidum10', 'I', 2)
(',', 'O', 2)
('by', 'O', 2)
('comparing', 'O', 0)
('the', 'O', -100)
('proliferative', 'O', 1)
('rate', 'O', -100)
('of', 'O', 2)
('human', 'O', 2)
('umbilical', 'O', 2)
('vein', 'O', 2)
('endothelial', 'O', 2)
('cells', 'O', 2)
('(', 'O', 2)
('HUVECs', 'O', 2)
(')', 'O', 2)
('exposed', 'O', 2)
('to', 'O', 2)
('recombinant', 'O', 2)
('TpF1', 'O', 2)
('or', 'O', 2)
('VEGF', 'O', 2)
('(', 'O', 2)
('free', 'O', 2)
('of', 'O', 2)
('Gram-positive', 'O', 2)
('and', 'O', -100)
('Gram-negative', 'O', 2)
('bacterial', 'O', 2)
('contaminants', 'O', 2)
(';', 'O', 2)
('Suppl', 'O', 2)
('.', 'O', 2)
('L.', 'B', -100)
('paracasei', 'I', 0)
('FAM18149', 'I', -100)
('

### Predict sentences again

In [8]:
ner = "ORGANISM"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=2)


In [10]:
import jsonlines

# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:

            # Get the value from the "text" field
            text = l["data"]["text"]
            
            # Pass the value to the nlp() function
            print(text)
            print(nlp(text))

The type strain, Sr54 ^T (=CGMCC 1.9249 ^T = NBRC 114034 ^T ), was isolated from the ice sample obtained from the supraglacial zone of No. 1 glacier in Xinjiang Uygur Autonomous Region, China.
[]
Organophosphorus hydrolase (OPH; EC 3.1.8.1), encoded by the identical opd genes from soil bacterium Pseudomonas diminuta MG and Flavobacterium sp.
[]
A human stool-derived Bilophila wadsworthia strain caused systemic inflammation in specific-pathogen-free mice (49).
[{'entity_group': 'I', 'score': 0.6806049, 'word': 'pathogen - free mice', 'start': 92, 'end': 110}]
The same applied to the potential donor E. coli 10716 which did not transfer plasmids to the three potential recipients Serratia marcescens subsp. marcescens DSM 30122, Enterobacter cloacae DSM 30060 and Proteus mirabilis DSM 4479.
[]
To further verify the function of c-di-GMP in controlling the switch between the two stages, a c-di-GMP phosphodiesterase (PDE), PA4781, ^28 was overexpressed in KT1115 and PAO1Δ wspF to test whether 

In [ ]:
kk